<a href="https://colab.research.google.com/github/bira1994/ADMN5015/blob/main/Heart_Attack_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
# Download training data
df_training = pd.read_csv("https://raw.githubusercontent.com/bira1994/ADMN5015/main/heart%20attack%20prediction%20training.csv")
df_training

,age,trtbps,chol,thalachh,oldpeak,output
0,56,120,236,178,0.8,1
1,57,120,354,163,0.6,1
2,57,140,192,148,0.4,1
3,56,140,294,153,1.3,1
4,44,120,263,173,0.0,1
...,...,...,...,...,...,...
295,57,140,241,123,0.2,0
296,45,110,264,132,1.2,0
297,68,144,193,141,3.4,0
298,57,130,131,115,1.2,0


In [4]:
# Make sure data types are correct (no objects)

df_training.dtypes

age           int64
trtbps        int64
chol          int64
thalachh      int64
oldpeak     float64
output        int64
dtype: object

In [5]:
# Pop label

target = df_training.pop('output')

In [17]:
# Define features

numeric_feature_names = ['age', 'trtbps', 'chol', 'thalachh','oldpeak']
numeric_features = df_training[numeric_feature_names]
numeric_features.head()

,age,trtbps,chol,thalachh,oldpeak
0,56,120,236,178,0.8
1,57,120,354,163,0.6
2,57,140,192,148,0.4
3,56,140,294,153,1.3
4,44,120,263,173,0.0


In [18]:
# Convert data to Tensorflow tensor

numeric_features = tf.convert_to_tensor(numeric_features)
numeric_features

<tf.Tensor: shape=(300, 5), dtype=float64, numpy=
array([[ 56. , 120. , 236. , 178. ,   0.8],
       [ 57. , 120. , 354. , 163. ,   0.6],
       [ 57. , 140. , 192. , 148. ,   0.4],
       ...,
       [ 68. , 144. , 193. , 141. ,   3.4],
       [ 57. , 130. , 131. , 115. ,   1.2],
       [ 57. , 130. , 236. , 174. ,   0. ]])>

In [19]:
# Normalize data

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)

In [34]:
# Define model

def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(2) # Code has only 2 outputs possibilities, therefore changing here
  ])

# Since my output has only 2 outcomes
# I tried using the BinaryCrossentropy but it was showing error in the next code as:
# "`logits` and `labels` must have the same shape, received ((100, 2) vs (100, 1))."
# Input & output nodes were mismatching
# Therefore using SparseCategoricalCrossentropy.
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                # loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [35]:
# Train model

BATCH_SIZE = 100

model = get_basic_model()
model.fit(numeric_features, target, epochs=1000, batch_size=BATCH_SIZE)
# Limited to 1000 epochs as after this number, 
# the rate of dec in Loss function & inc in accuracy highly reduced

Epoch 1/1000
3/3 [==============================] - 0s 4ms/step - loss: 0.7008 - accuracy: 0.5267
Epoch 2/1000
3/3 [==============================] - 0s 5ms/step - loss: 0.6943 - accuracy: 0.5300
Epoch 3/1000
3/3 [==============================] - 0s 4ms/step - loss: 0.6880 - accuracy: 0.5467
Epoch 4/1000
3/3 [==============================] - 0s 4ms/step - loss: 0.6831 - accuracy: 0.5600
Epoch 5/1000
3/3 [==============================] - 0s 3ms/step - loss: 0.6781 - accuracy: 0.5700
Epoch 6/1000
3/3 [==============================] - 0s 5ms/step - loss: 0.6731 - accuracy: 0.5733
Epoch 7/1000
3/3 [==============================] - 0s 7ms/step - loss: 0.6686 - accuracy: 0.5867
Epoch 8/1000
3/3 [==============================] - 0s 4ms/step - loss: 0.6645 - accuracy: 0.5933
Epoch 9/1000
3/3 [==============================] - 0s 3ms/step - loss: 0.6605 - accuracy: 0.5900
Epoch 10/1000
3/3 [==============================] - 0s 4ms/step - loss: 0.6569 - accuracy: 0.5833
Epoch 11/1000
3/3 [

In [36]:
# Download test data

df_test = pd.read_csv("https://raw.githubusercontent.com/bira1994/ADMN5015/main/heart%20attack%20prediction%20testing.csv")
df_test.head()

,age,trtbps,chol,thalachh,oldpeak,output
0,56,120,236,178,0.8,1
1,57,120,354,163,0.6,1
2,57,140,192,148,0.4,1
3,56,140,294,153,1.3,1
4,44,120,263,173,0.0,1


In [37]:
# Pop label

test_target = df_test.pop('output')

In [39]:
# Define test features

test_numeric_feature_names = ['age','trtbps', 'chol', 'thalachh', 'oldpeak']
test_numeric_features = df_test[test_numeric_feature_names]
test_numeric_features.head()

,age,trtbps,chol,thalachh,oldpeak
0,56,120,236,178,0.8
1,57,120,354,163,0.6
2,57,140,192,148,0.4
3,56,140,294,153,1.3
4,44,120,263,173,0.0


In [40]:
# Convert test data to Tensorflow tensor

test_numeric_features = tf.convert_to_tensor(test_numeric_features)
test_numeric_features

<tf.Tensor: shape=(200, 5), dtype=float64, numpy=
array([[5.60e+01, 1.20e+02, 2.36e+02, 1.78e+02, 8.00e-01],
       [5.70e+01, 1.20e+02, 3.54e+02, 1.63e+02, 6.00e-01],
       [5.70e+01, 1.40e+02, 1.92e+02, 1.48e+02, 4.00e-01],
       [5.60e+01, 1.40e+02, 2.94e+02, 1.53e+02, 1.30e+00],
       [4.40e+01, 1.20e+02, 2.63e+02, 1.73e+02, 0.00e+00],
       [5.20e+01, 1.72e+02, 1.99e+02, 1.62e+02, 5.00e-01],
       [5.70e+01, 1.50e+02, 1.68e+02, 1.74e+02, 1.60e+00],
       [5.40e+01, 1.40e+02, 2.39e+02, 1.60e+02, 1.20e+00],
       [4.80e+01, 1.30e+02, 2.75e+02, 1.39e+02, 2.00e-01],
       [4.90e+01, 1.30e+02, 2.66e+02, 1.71e+02, 6.00e-01],
       [6.40e+01, 1.10e+02, 2.11e+02, 1.44e+02, 1.80e+00],
       [5.80e+01, 1.50e+02, 2.83e+02, 1.62e+02, 1.00e+00],
       [5.00e+01, 1.20e+02, 2.19e+02, 1.58e+02, 1.60e+00],
       [5.80e+01, 1.20e+02, 3.40e+02, 1.72e+02, 0.00e+00],
       [6.60e+01, 1.50e+02, 2.26e+02, 1.14e+02, 2.60e+00],
       [4.30e+01, 1.50e+02, 2.47e+02, 1.71e+02, 1.50e+00],
      

In [41]:
# Normalize data

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(test_numeric_features)

In [42]:
# Evaluate test data based on training data

score = model.evaluate(test_numeric_features, test_target, verbose=1)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

7/7 [==============================] - 0s 2ms/step - loss: 0.3442 - accuracy: 0.8550
Test loss: 0.34423157572746277 / Test accuracy: 0.8550000190734863


In [ ]:
# Save model (optional)

# model.save('iris_model')

In [44]:
# Download cases to be predicted

df_predict = pd.read_csv("https://raw.githubusercontent.com/bira1994/ADMN5015/main/heart%20attack%20to%20predict.csv")
df_predict

,age,trtbps,chol,thalachh,oldpeak
0,63,145,233,150,2.3
1,37,130,250,187,3.5
2,41,130,204,172,1.4


In [45]:
# Make sure data types are correct (no objects)

df_predict.dtypes

age           int64
trtbps        int64
chol          int64
thalachh      int64
oldpeak     float64
dtype: object

In [46]:
# Convert data to Tensorflow tensor

predict_numeric_features = tf.convert_to_tensor(df_predict)
predict_numeric_features

<tf.Tensor: shape=(3, 5), dtype=float64, numpy=
array([[ 63. , 145. , 233. , 150. ,   2.3],
       [ 37. , 130. , 250. , 187. ,   3.5],
       [ 41. , 130. , 204. , 172. ,   1.4]])>

In [47]:
# Normalize

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(predict_numeric_features)

In [48]:
# Predict labels

class_names = ['Low chance of heart attack', 'high chance of heart attack']

predictions = model(predict_numeric_features, training=False)

# Create new columns in dataframe
df_predict['label'] = None
df_predict['certainty'] = None

for i, logits in enumerate(predictions):
  class_idx = tf.argmax(logits).numpy()
  p = tf.nn.softmax(logits)[class_idx]
  name = class_names[class_idx]
  print("Example {} prediction: {} ({:4.1f}%)".format(i, name, 100*p))

  # Save predictions to dataframe
  df_predict["label"].iloc[i] = name
  df_predict['certainty'].iloc[i] = format(p)


Example 0 prediction: high chance of heart attack (57.1%)
Example 1 prediction: Low chance of heart attack (64.6%)
Example 2 prediction: high chance of heart attack (96.5%)


In [49]:
df_predict

,age,trtbps,chol,thalachh,oldpeak,label,certainty
0,63,145,233,150,2.3,high chance of heart attack,0.5714207887649536
1,37,130,250,187,3.5,Low chance of heart attack,0.6461160778999329
2,41,130,204,172,1.4,high chance of heart attack,0.9645413160324097


The tool that has been created will be very useful in prediction of heart attacks & can be used as an early guidance system for patients, helping them to be  better informed & make changes in their lifestyle before aything major happens. 